In [1]:
import numpy as np
import scipy.stats as stats
import pandas as pd
import polars as pl
import matplotlib.pyplot as plt
# from dynamic_routing_analysis import decoding_utils
# from dynamic_routing_analysis import plot_utils
# import dynamic_routing_analysis as dra
import pingouin as pg
import os

import matplotlib
import matplotlib.font_manager as fm

matplotlib.rcParams['font.size'] = 8
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42
fm.FontProperties().set_family('arial')

%load_ext autoreload
%autoreload 2
# %matplotlib inline
%matplotlib widget

In [58]:
results_path="s3://aind-scratch-data/dynamic-routing/decoding/results/keep_units_consistent_test_20250605_170828/"

In [3]:
session_table_path=r"\\allen\programs\mindscope\workgroups\dynamicrouting\Ethan\CO decoding results\session_table_v0.265.csv"
session_table=pl.read_csv(session_table_path)

dr_session_list=(
    session_table.filter(
    pl.col('project')=="DynamicRouting",
    pl.col('is_production'),
    pl.col('is_annotated'),
    pl.col('issues')=="",
    # pl.col('is_good_behavior').eq(True),
    )['session_id'].to_list()
    )

In [ ]:
#define grouping columns
grouping_cols = {
    'session_id',
    'structure',
    'electrode_group_names',
    'unit_subsample_size',
    'bin_center',
    'bin_size',
    'time_aligned_to',
    'coefs',
    'unit_ids',
}

#toggle combine_multi_probe_rec
combine_multi_probe_rec = True

if combine_multi_probe_rec:
    combine_multi_probe_expr = pl.col('electrode_group_names').list.len().gt(1) | pl.col('is_sole_recording').eq(True)
else:
    combine_multi_probe_expr = pl.col('electrode_group_names').list.len().eq(1) | pl.col('is_sole_recording').eq(True)


In [59]:
#get single session, keep coefs (and unit ids?)

#include repeats, but don't need predict_proba

#one row per repeat, with coefs and unit ids

### make separate query to include repeats - specify session
sel_session='742903_2024-10-22' #the golden session
# sel_session='670180_2023-07-27' #a templeton session
decoder_coefs = (
    pl.scan_parquet(results_path)
    .with_columns(
        pl.col('electrode_group_names').flatten().n_unique().eq(1).over(grouping_cols - {'electrode_group_names'}).alias('is_sole_recording'),     
    )
    .filter(
        combine_multi_probe_expr,
        pl.col('is_all_trials').not_(),
        pl.col('session_id').eq(sel_session),
        # pl.col('shift_idx').eq(0)
    )
    .drop('electrode_group_names', 'is_all_trials', 'is_sole_recording')
    # .group_by(grouping_cols | {'shift_idx'}, maintain_order=True)
    # .agg(
    #     pl.col('balanced_accuracy_test').mean(), # over repeats
    # )
    # # get the aligned result and median over shifts:
    # .group_by(grouping_cols - {'electrode_group_names'})
    # .agg(
    #     pl.col('balanced_accuracy_test').filter(pl.col('shift_idx') == 0).first().alias('mean_true'),
    #     pl.col('balanced_accuracy_test').filter(pl.col('shift_idx') != 0).median().alias('median_null'),
    #     pl.col('balanced_accuracy_test', 'shift_idx').sort_by('shift_idx'),
    # )
    # # get the difference between true and null:
    # .with_columns(
    #     pl.col('mean_true').sub(pl.col('median_null')).alias('mean_diff'),
    # )
    .sort('structure', 'shift_idx', 'repeat_idx', 'time_aligned_to', 'bin_center', descending=False, maintain_order=True)
    .collect()

    # .drop('shift_idx', 'is_all_trials', 'electrode_group_names', 'unit_criteria', 'is_sole_recording')
    # .group_by('session_id', 'structure', 'unit_subsample_size', 'repeat_idx')
    # .agg(
    #     pl.col('balanced_accuracy_test', 'total_n_units').first(),
    #     pl.col('predict_proba', 'trial_index', 'is_vis_context', 'stim_name', 'is_response', 'trial_index_in_block', 'block_index'),
    # )
    # .sort('session_id','structure', 'unit_subsample_size', 'repeat_idx')
    # # .group_by('session_id','structure')
    # .collect()
)

In [61]:
decoder_coefs

balanced_accuracy_test,balanced_accuracy_train,time_aligned_to,bin_size,bin_center,shift_idx,repeat_idx,predict_proba,trial_indices,unit_ids,coefs,session_id,structure,unit_subsample_size,unit_criteria
f64,f64,enum,f64,f64,i16,u16,list[f64],list[u16],list[str],list[f64],str,str,u16,str
0.583333,0.640945,"""stim_start_time""",0.5,-1.25,-43,0,null,null,"[""742903_2024-10-22_A-119"", ""742903_2024-10-22_A-367"", … ""742903_2024-10-22_A-345""]","[0.054951, 0.209828, … 0.118389]","""742903_2024-10-22""","""ACAd""",20,"""loose_drift"""
0.559034,0.641427,"""stim_start_time""",0.5,-0.75,-43,0,null,null,"[""742903_2024-10-22_A-127"", ""742903_2024-10-22_A-365"", … ""742903_2024-10-22_A-351""]","[0.330015, -0.142493, … -0.238176]","""742903_2024-10-22""","""ACAd""",20,"""loose_drift"""
0.614203,0.653527,"""stim_start_time""",0.5,-0.25,-43,0,null,null,"[""742903_2024-10-22_A-373"", ""742903_2024-10-22_A-117"", … ""742903_2024-10-22_A-567""]","[0.081674, -0.095195, … -0.123481]","""742903_2024-10-22""","""ACAd""",20,"""loose_drift"""
0.561304,0.62368,"""stim_start_time""",0.5,0.25,-43,0,null,null,"[""742903_2024-10-22_A-351"", ""742903_2024-10-22_A-567"", … ""742903_2024-10-22_A-326""]","[-0.214795, 0.531285, … -0.310547]","""742903_2024-10-22""","""ACAd""",20,"""loose_drift"""
0.571932,0.646289,"""stim_start_time""",0.5,0.75,-43,0,null,null,"[""742903_2024-10-22_A-567"", ""742903_2024-10-22_A-352"", … ""742903_2024-10-22_A-107""]","[0.189186, 0.207, … 0.179007]","""742903_2024-10-22""","""ACAd""",20,"""loose_drift"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
0.503285,0.563781,"""stim_start_time""",0.5,3.25,44,24,null,null,"[""742903_2024-10-22_E-557"", ""742903_2024-10-22_E-1005"", … ""742903_2024-10-22_E-33""]","[0.251334, 0.432435, … -0.246885]","""742903_2024-10-22""","""TTd""",20,"""loose_drift"""
0.534396,0.591421,"""stim_start_time""",0.5,3.75,44,24,null,null,"[""742903_2024-10-22_E-1005"", ""742903_2024-10-22_E-564"", … ""742903_2024-10-22_E-18""]","[-0.032576, -0.331939, … 0.017552]","""742903_2024-10-22""","""TTd""",20,"""loose_drift"""
0.5143,0.605835,"""stim_start_time""",0.5,4.25,44,24,null,null,"[""742903_2024-10-22_E-18"", ""742903_2024-10-22_E-557"", … ""742903_2024-10-22_E-535""]","[0.201722, -0.216877, … -0.11906]","""742903_2024-10-22""","""TTd""",20,"""loose_drift"""


In [68]:
decoder_coefs_pd=decoder_coefs.to_pandas()

In [70]:
savepath=r"\\allen\programs\mindscope\workgroups\dynamicrouting\Ethan\CO decoding results\keep_units_consistent_test_2025-06-05_170828\decoder_coefs.parquet"
decoder_coefs_pd.to_parquet(savepath)

In [63]:
# for each repeat, 

# get the balanced accuracy test & train, plus the coefs and unit ids across timebins

# check the unit ids are the same across all timebins

# compute the correlation of the coefs to the first timebin


# start with one repeat, one structure

sel_structure='ACAd'
sel_repeat=0
sel_shift_idx=0
# sel_time_bin=-0.25
sel_time_aligned_to='stim_start_time'


In [34]:
# decoder_coefs_pd.query(f'structure == "{sel_structure}" and repeat_idx == {sel_repeat} and time_aligned_to == "{sel_time_aligned_to}"')['unit_ids']
# decoder_coefs_pd.query(f'structure == "{sel_structure}" and bin_center == {sel_time_bin}')

In [64]:
# unit_ids=decoder_coefs_pd.query(f'structure == "{sel_structure}" and repeat_idx == {sel_repeat} and time_aligned_to == "{sel_time_aligned_to}"')['unit_ids']

# unit_ids=decoder_coefs_pd.query(f'structure == "{sel_structure}" and bin_center == {sel_time_bin} and \
#                                 repeat_idx == {sel_repeat} and time_aligned_to == "{sel_time_aligned_to}"')['unit_ids'].values

unit_ids=decoder_coefs_pd.query(f'structure == "{sel_structure}" and shift_idx == {sel_shift_idx} and \
                                repeat_idx == {sel_repeat} and time_aligned_to == "{sel_time_aligned_to}"')['unit_ids'].values

sorted_unit_ids=[]

for uu in unit_ids:
    sorted_unit_ids.append(np.sort(uu))

sorted_unit_ids==sorted_unit_ids[0]

array([[ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True],
       [ T

In [57]:
subsample=np.array([1,3,7,13,20])
full_array=np.arange(0,30)

np.where(np.isin(full_array, subsample))[0]


array([ 1,  3,  7, 13, 20], dtype=int64)